In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity


import numpy as np
import re



In [5]:
def clean_filename(url):
    return re.sub(r'[\/:*?"<>|]', '_', url)

In [6]:
with open("urls.txt", "r") as f:
    url_list = f.read().splitlines()


for url in url_list:
    response = requests.get(url)
    if response.status_code == 200:
        html_content = response.text  
    else:
        print(f"Failed to retrieve {url}")
    html_content  
    soup = BeautifulSoup(html_content, 'html.parser')
    paragraphs = soup.find_all('p')
    article_text = ' '.join([para.get_text() for para in paragraphs])

    with open(f"scraped_article_{clean_filename(url)}.txt", "w") as f:
        f.write(article_text)

    nltk.download('punkt')
    nltk.download('stopwords')

    with open(f"scraped_article_{clean_filename(url)}.txt", "r") as f:
        article_text = f.read()

    sentences = sent_tokenize(article_text)

    stop_words = set(stopwords.words('english'))
    cleaned_sentences_nltk = []

    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        words = [word for word in words if word not in stop_words and word not in string.punctuation]
        cleaned_sentences_nltk.append(' '.join(words))

    print(cleaned_sentences_nltk)
    cvect = CountVectorizer(ngram_range=(1,1), token_pattern='(?u)\\b\\w+\\b')
    counts = cvect.fit_transform(cleaned_sentences_nltk)
    normalized_counts = normalize(counts, norm='l1', axis=1)

    tfidf = TfidfVectorizer(ngram_range=(1,1))
    tfs = tfidf.fit_transform(cleaned_sentences_nltk)

    sentence_scores = np.sum(tfs.toarray(), axis=1)

    num_sentences = 5
    top_sentence_indices = np.argsort(sentence_scores)[-num_sentences:]

    # Sélection des phrases les plus importantes
    summary = [sentences[i] for i in sorted(top_sentence_indices)]

    
    vectorizer = CountVectorizer().fit_transform(summary)
    vectors = vectorizer.toarray()
    cosine_matrix = cosine_similarity(vectors)

    threshold = 0.7
    redundant_sentences = set()

    for i in range(len(cosine_matrix)):
        for j in range(i + 1, len(cosine_matrix)):
            if cosine_matrix[i][j] > threshold:
                redundant_sentences.add(j)


    filtered_summary = [sentence for idx, sentence in enumerate(summary) if idx not in redundant_sentences]
    
    with open(f"summary_{clean_filename(url)}.txt", "w") as f:
        for sentence in filtered_summary:
            f.write(sentence + "\n")
        f.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['markets hot stocks fear greed index latest market news hot stocks ghost shark manta ray protect undersea realm', 'sounds like plot future marvel movie actual fact ’ could future pacific naval defenses', 'ghost shark manta ray names prototype uncrewed underwater vehicles – uuvs drones – introduced recently australia united states respectively', 'experts say submersibles could represent future undersea warfare showing ability exert power minimizing danger human life', 'use drones aerial warfare become commonplace', 'us used extensively conflicts iraq afghanistan beginning 1990s newer cheaper drones become key pieces military hardware sides russia ’ invasion ukraine', 'kyiv also built naval surface drones inflicted heavy losses much larger expensive ships russia ’ black sea fleet', 'aerial surface sea drones controlled using satellites light radio waves', '’ function way depths', '2023 study published swiss journal sensors points underwater communications require energy still see signif

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
